In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv('Daily+temperature+data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df.head()

,Temperature(Min)
Date,
2017-01-01,14.3
2017-01-02,17.4
2017-01-03,18.5
2017-01-04,16.8
2017-01-05,11.5


In [14]:
train_len=511 
train = df[:train_len]
test = df[train_len:]

### Simple Exponential Smoothing

In [15]:
len(test)

219

In [16]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
model = SimpleExpSmoothing(train['Temperature(Min)'])
model_fit = model.fit(smoothing_level=0.2, optimized=False)
y_hat_ses = test.copy()
y_hat_ses['ses_forecast'] = model_fit.forecast(219)

In [18]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(np.mean(mean_squared_error(test['Temperature(Min)'],y_hat_ses['ses_forecast']))).round(2)
mape = np.round((np.mean(np.abs(test['Temperature(Min)']-y_hat_ses['ses_forecast'])/test['Temperature(Min)']))*100,2)
rmse,mape

(3.56, 30.24)

### Holts Exponential Smoothing

In [19]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
model = ExponentialSmoothing(np.asarray(train['Temperature(Min)']),seasonal_periods=12,trend='additive',seasonal = None)
model_fit = model.fit(optimized=False,smoothing_level=0.5,smoothing_slope=0.01)
print(model_fit.params)
y_hat_hes = test.copy()
y_hat_hes['hes_forecast'] = model_fit.forecast(len(test))

{'smoothing_level': 0.5, 'smoothing_trend': 0.01, 'smoothing_seasonal': None, 'damping_trend': nan, 'initial_level': 16.039999999999992, 'initial_trend': -0.26545454545454444, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


In [20]:
rmse = np.sqrt(np.mean(mean_squared_error(test['Temperature(Min)'],y_hat_hes['hes_forecast']))).round(2)
mape = np.round((np.mean(np.abs(test['Temperature(Min)']-y_hat_hes['hes_forecast'])/test['Temperature(Min)']))*100,2)
rmse,mape

(5.95, 41.94)

### Holt Winters Smoothing

In [21]:
model =ExponentialSmoothing(np.asarray(train['Temperature(Min)']),seasonal_periods=30,trend='add',seasonal='add')
model_fit = model.fit(optimized=True)
print(model_fit.params)
y_hat_hwa = test.copy()
y_hat_hwa['hwa_forecast'] = model_fit.forecast(len(test))

{'smoothing_level': 0.6742905531408826, 'smoothing_trend': 0.0, 'smoothing_seasonal': 0.0, 'damping_trend': nan, 'initial_level': 14.940153469593199, 'initial_trend': -0.009875626133881025, 'initial_seasons': array([ 0.52726187,  0.03275887,  0.0546689 ,  0.56433055,  0.03900888,
       -0.13932503, -0.02944389,  0.62153047,  0.77264519,  0.45323589,
        0.53353392,  0.27878707, -0.71724563,  0.23388172,  0.43173867,
        0.4594823 , -0.58961221,  0.22015795, -0.0816295 , -0.76005189,
       -0.53853719,  0.29500932, -0.87756244, -1.57357523, -0.71076678,
       -0.70671762, -0.43816353,  0.17757992,  0.69326927,  0.70914971]), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


In [22]:
rmse = np.sqrt(np.mean(mean_squared_error(test['Temperature(Min)'],y_hat_hwa['hwa_forecast']))).round(2)
mape = np.round((np.mean(np.abs(test['Temperature(Min)']-y_hat_hwa['hwa_forecast'])/test['Temperature(Min)']))*100,2)
rmse,mape

(4.14, 34.37)